In [2]:
import pandas as pd
import numpy as np  
import seaborn as sns
import matplotlib.pyplot as pyplot
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import nltk
import re
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
import string
from wordcloud import STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier



In [3]:
df  = pd.read_pickle("df_balanced.pkl")


In [4]:
df.head()

,id,keyword,location,text,target,presence_location,presence_keyword,char_count,word_count,stopword_count,...,keyword_tokens,location_tokens,text_tokens,text_stem,text_lemma,keyword_stem,keyword_lemma,location_stem,location_lemma,text_combined
6818,9765,trapped,10 Steps Ahead. Cloud 9,Bomb head? Explosive decisions dat produced mo...,1,1,1,134,21,6,...,[trapped],"[steps, ahead, cloud]","[bomb, head, explosive, decisions, dat, produc...",bomb head explos decis dat produc dead childre...,bomb head explosive decision dat produced dead...,trap,trapped,step ahead cloud,step ahead cloud,bomb head explosive decision dat produced dead...
4842,6896,mass%20murder,"Huntsville, AL",Okay not sure the word 'mass murder' applies d...,1,1,1,121,19,9,...,[massmurder],"[huntsville, al]","[okay, sure, word, mass, murder, applies, war,...",okay sure word mass murder appli war horrend n...,okay sure word mass murder applies war horrend...,massmurd,massmurder,huntsvill al,huntsville al,okay sure word mass murder applies war horrend...
5848,8356,ruin,Garrett,like why on earth would you want anybody to be...,0,1,1,101,17,6,...,[ruin],[garrett],"[like, earth, would, want, anybody, unhappy, d...",like earth would want anybodi unhappi dont pur...,like earth would want anybody unhappy dont pur...,ruin,ruin,garrett,garrett,like earth would want anybody unhappy dont pur...
1851,2661,crush,"Cleveland, Ohio",My woman crush wedneday goes to the beautiful ...,0,1,1,108,12,3,...,[crush],"[cleveland, ohio]","[woman, crush, wedneday, goes, beautiful]",woman crush wedneday goe beauti,woman crush wedneday go beautiful,crush,crush,cleveland ohio,cleveland ohio,woman crush wedneday go beautiful crush clevel...
5729,8176,rescuers,Washington,#News: 'Many deaths' in shipwreck: Rescuers ar...,1,1,1,136,18,6,...,[rescuers],[washington],"[many, deaths, shipwreck, rescuers, trying, sa...",mani death shipwreck rescuer tri save hundr mi...,many death shipwreck rescuer trying save hundr...,rescuer,rescuer,washington,washington,many death shipwreck rescuer trying save hundr...


In [ ]:
df["has_question"] = df["text"].str.contains(r"\?").astype(int)
df["exclamation_count"] = df["text"].str.count(r"!").astype(int)
### A rajouter dans 3 EDA 

In [ ]:
# Par exemple pour lemmatized
df["combined_lemma"] = (
    df["text_lemma"] + " " +
    df["keyword_lemma"].fillna("") + " " +
    df["location_lemma"].fillna("")
)

# Pour stemmed
df["combined_stemmed"] = (
    df["text_stem"] + " " +
    df["keyword_stem"].fillna("") + " " +
    df["location_stem"].fillna("")
)



In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4392 entries, 6818 to 2019
Data columns (total 35 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 4392 non-null   int64  
 1   keyword            4392 non-null   object 
 2   location           4392 non-null   object 
 3   text               4392 non-null   object 
 4   target             4392 non-null   int64  
 5   presence_location  4392 non-null   int64  
 6   presence_keyword   4392 non-null   int64  
 7   char_count         4392 non-null   int64  
 8   word_count         4392 non-null   int64  
 9   stopword_count     4392 non-null   int64  
 10  unique_word_count  4392 non-null   int64  
 11  stop_word_count    4392 non-null   int64  
 12  url_count          4392 non-null   int64  
 13  mean_word_length   4392 non-null   float64
 14  punctuation_count  4392 non-null   int64  
 15  hashtag_count      4392 non-null   int64  
 16  mention_count      4392 no

In [8]:
df.to_csv("df_balanced.csv", index=False)


In [9]:
df.head()

,id,keyword,location,text,target,presence_location,presence_keyword,char_count,word_count,stopword_count,...,text_lemma,keyword_stem,keyword_lemma,location_stem,location_lemma,text_combined,has_question,exclamation_count,combined_lemma,combined_stemmed
6818,9765,trapped,10 Steps Ahead. Cloud 9,Bomb head? Explosive decisions dat produced mo...,1,1,1,134,21,6,...,bomb head explosive decision dat produced dead...,trap,trapped,step ahead cloud,step ahead cloud,bomb head explosive decision dat produced dead...,1,0,bomb head explosive decision dat produced dead...,bomb head explos decis dat produc dead childre...
4842,6896,mass%20murder,"Huntsville, AL",Okay not sure the word 'mass murder' applies d...,1,1,1,121,19,9,...,okay sure word mass murder applies war horrend...,massmurd,massmurder,huntsvill al,huntsville al,okay sure word mass murder applies war horrend...,0,0,okay sure word mass murder applies war horrend...,okay sure word mass murder appli war horrend n...
5848,8356,ruin,Garrett,like why on earth would you want anybody to be...,0,1,1,101,17,6,...,like earth would want anybody unhappy dont pur...,ruin,ruin,garrett,garrett,like earth would want anybody unhappy dont pur...,0,0,like earth would want anybody unhappy dont pur...,like earth would want anybodi unhappi dont pur...
1851,2661,crush,"Cleveland, Ohio",My woman crush wedneday goes to the beautiful ...,0,1,1,108,12,3,...,woman crush wedneday go beautiful,crush,crush,cleveland ohio,cleveland ohio,woman crush wedneday go beautiful crush clevel...,0,0,woman crush wedneday go beautiful crush clevel...,woman crush wedneday goe beauti crush clevelan...
5729,8176,rescuers,Washington,#News: 'Many deaths' in shipwreck: Rescuers ar...,1,1,1,136,18,6,...,many death shipwreck rescuer trying save hundr...,rescuer,rescuer,washington,washington,many death shipwreck rescuer trying save hundr...,0,0,many death shipwreck rescuer trying save hundr...,mani death shipwreck rescuer tri save hundr mi...


In [23]:
TEXT_COLUMNS = {
    "clean_text": "token", 
    "lemma": "text_lemma",                          
    "combined_lemma": "combined_lemma",
    "stemmed": "text_stem",                       
    "combined_stemmed": "combined_stemmed"
}



# Méta-features numériques à tester
META_FEATURES = ["word_count", "stopword_count", "url_count", "char_count", "punctuation_count", "hashtag_count", "mention_count" , "has_question", "exclamation_count", "presence_keyword", "presence_location"]

# Modèles à tester
MODELS = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(),
    "SVM": SVC(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
import pandas as pd

# Dictionnaires
TEXT_COLUMNS = {
    "clean_text": "token", 
    "lemma": "text_lemma",                          
    "combined_lemma": "combined_lemma",
    "stemmed": "text_stem",                       
    "combined_stemmed": "combined_stemmed"
}

META_FEATURES = [
    "word_count", "stopword_count", "url_count", "char_count",
    "punctuation_count", "hashtag_count", "mention_count",
    "has_question", "exclamation_count", "presence_keyword", "presence_location"
]

MODELS = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(),
    "SVM": SVC(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

results = []

# Boucles principales
for text_name, text_col in TEXT_COLUMNS.items():
    for use_meta in [False, True]:
        selected_features = [text_col] + (META_FEATURES if use_meta else [])
        missing = [col for col in selected_features if col not in df.columns]
        if missing:
            continue

        X = df[selected_features]
        y = df["target"]
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

        for model_name, model in MODELS.items():
            transformers = [("tfidf", TfidfVectorizer(max_features=3000, ngram_range=(1,2)), text_col)]
            if use_meta:
                transformers.append(("num", StandardScaler(), META_FEATURES))

            preprocessor = ColumnTransformer(transformers)

            pipe = Pipeline([
                ("preprocessing", preprocessor),
                ("clf", model)
            ])

            pipe.fit(X_train, y_train)
            y_pred = pipe.predict(X_test)

            accuracy = accuracy_score(y_test, y_pred)
            precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None, labels=[0, 1])
            results.append({
                "text_column": text_name,
                "use_meta": use_meta,
                "model": model_name,
                "recall_1": recall[1],
                "f1_1": f1[1],
                "accuracy": accuracy
            })

# Résumé final
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by="recall_1", ascending=False)
display(df_results[["text_column", "use_meta", "model", "recall_1", "f1_1", "accuracy"]])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:15:20] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:15:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:15:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:15:25] WARN

,text_column,use_meta,model,recall_1,f1_1,accuracy
8,combined_lemma,False,LogisticRegression,0.776765,0.780320,0.781570
0,lemma,False,LogisticRegression,0.772210,0.777523,0.779295
24,combined_stemmed,False,LogisticRegression,0.751708,0.765661,0.770193
16,stemmed,False,LogisticRegression,0.749431,0.758939,0.762230
2,lemma,False,SVM,0.749431,0.766007,0.771331
4,lemma,True,LogisticRegression,0.744875,0.770318,0.778157
10,combined_lemma,False,SVM,0.744875,0.765808,0.772469
12,combined_lemma,True,LogisticRegression,0.740319,0.763807,0.771331
26,combined_stemmed,False,SVM,0.740319,0.760234,0.766780
20,stemmed,True,LogisticRegression,0.740319,0.769231,0.778157


In [12]:
pd.set_option("display.max_rows", None)
display(df_results[["text_column", "use_meta", "model", "recall_1", "f1_1", "accuracy"]])


,text_column,use_meta,model,recall_1,f1_1,accuracy
16,combined_lemma,False,LogisticRegression,0.776765,0.780320,0.781570
8,lemma,False,LogisticRegression,0.772210,0.777523,0.779295
32,combined_stemmed,False,LogisticRegression,0.751708,0.765661,0.770193
24,stemmed,False,LogisticRegression,0.749431,0.758939,0.762230
10,lemma,False,SVM,0.749431,0.766007,0.771331
4,clean_text,True,LogisticRegression,0.747153,0.773585,0.781570
12,lemma,True,LogisticRegression,0.744875,0.770318,0.778157
18,combined_lemma,False,SVM,0.744875,0.765808,0.772469
20,combined_lemma,True,LogisticRegression,0.740319,0.763807,0.771331
34,combined_stemmed,False,SVM,0.740319,0.760234,0.766780


In [13]:
def get_strategy(model_name):
    if model_name == "RandomForest":
        return "Bagging"
    elif model_name == "XGBoost":
        return "Boosting"
    else:
        return "None"

df_results["strategy"] = df_results["model"].apply(get_strategy)


In [14]:
df_results.groupby("strategy")[["recall_1", "f1_1", "accuracy"]].mean().sort_values(by="recall_1", ascending=False)


,recall_1,f1_1,accuracy
strategy,,,
None,0.719248,0.743539,0.75273
Bagging,0.679499,0.729134,0.74835
Boosting,0.676310,0.727111,0.74653


In [15]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# Charger le fichier CSV
df = pd.read_csv("df_balanced.csv")
y = df["target"]

# Colonnes texte à tester
TEXT_COLUMNS = {
    "clean_text": "clean_text", 
    "lemma": "text_lemma",                          
    "combined_lemma": "combined_lemma",
    "stemmed": "text_stem",                       
    "combined_stemmed": "combined_stemmed"
}

# Méta-features
META_FEATURES = [
    "word_count", "stopword_count", "url_count", "char_count",
    "punctuation_count", "hashtag_count", "mention_count",
    "has_question", "exclamation_count", "presence_keyword", "presence_location"
]

# Modèles et grilles de paramètres
MODELS_PARAMS = {
    "LogisticRegression": (
        LogisticRegression(max_iter=1000),
        {"clf__C": [0.1, 1, 10]}
    ),
    "RandomForest": (
        RandomForestClassifier(),
        {"clf__n_estimators": [50, 100], "clf__max_depth": [None, 10]}
    ),
    "SVM": (
        SVC(),
        {"clf__C": [0.1, 1, 10]}
    ),
    "XGBoost": (
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        {"clf__n_estimators": [50, 100], "clf__max_depth": [3, 6]}
    )
}

results = []

# Boucle sur les colonnes texte et sur les méta-features
for text_name, text_col in TEXT_COLUMNS.items():
    for use_meta in [False, True]:
        selected_features = [text_col] + (META_FEATURES if use_meta else [])
        if not all(col in df.columns for col in selected_features):
            continue

        X = df[selected_features]
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

        for model_name, (model, param_grid) in MODELS_PARAMS.items():
            print(f"🔍 GridSearch - Text: {text_name} | Meta: {use_meta} | Model: {model_name}")

            transformers = [("tfidf", TfidfVectorizer(max_features=3000, ngram_range=(1,2)), text_col)]
            if use_meta:
                transformers.append(("num", StandardScaler(), META_FEATURES))

            preprocessor = ColumnTransformer(transformers)
            pipe = Pipeline([
                ("preprocessing", preprocessor),
                ("clf", model)
            ])

            grid = GridSearchCV(pipe, param_grid, cv=3, scoring='f1', n_jobs=-1)
            grid.fit(X_train, y_train)
            y_pred = grid.predict(X_test)

            score = classification_report(y_test, y_pred, output_dict=True)
            results.append({
                "text_column": text_name,
                "use_meta": use_meta,
                "model": model_name,
                "best_params": grid.best_params_,
                "accuracy": score["accuracy"],
                "recall_1": score["1"]["recall"],
                "f1_1": score["1"]["f1-score"]
            })

# Résumé
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by="f1_1", ascending=False)
df_results.to_csv("gridsearch_results.csv", index=False)
print("\n✅ Résultats sauvegardés dans gridsearch_results.csv")


🔍 GridSearch - Text: clean_text | Meta: False | Model: LogisticRegression
🔍 GridSearch - Text: clean_text | Meta: False | Model: RandomForest
🔍 GridSearch - Text: clean_text | Meta: False | Model: RandomForest
🔍 GridSearch - Text: clean_text | Meta: False | Model: SVM
🔍 GridSearch - Text: clean_text | Meta: False | Model: SVM
🔍 GridSearch - Text: clean_text | Meta: False | Model: XGBoost
🔍 GridSearch - Text: clean_text | Meta: False | Model: XGBoost


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:19:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:19:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:19:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:19:33] WARN

🔍 GridSearch - Text: clean_text | Meta: True | Model: LogisticRegression
🔍 GridSearch - Text: clean_text | Meta: True | Model: RandomForest
🔍 GridSearch - Text: clean_text | Meta: True | Model: RandomForest
🔍 GridSearch - Text: clean_text | Meta: True | Model: SVM
🔍 GridSearch - Text: clean_text | Meta: True | Model: SVM
🔍 GridSearch - Text: clean_text | Meta: True | Model: XGBoost
🔍 GridSearch - Text: clean_text | Meta: True | Model: XGBoost


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:19:40] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:19:40] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:19:40] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:19:40] WARN

🔍 GridSearch - Text: lemma | Meta: False | Model: LogisticRegression
🔍 GridSearch - Text: lemma | Meta: False | Model: RandomForest
🔍 GridSearch - Text: lemma | Meta: False | Model: SVM
🔍 GridSearch - Text: lemma | Meta: False | Model: SVM
🔍 GridSearch - Text: lemma | Meta: False | Model: XGBoost
🔍 GridSearch - Text: lemma | Meta: False | Model: XGBoost


...

🔍 GridSearch - Text: combined_stemmed | Meta: False | Model: LogisticRegression
🔍 GridSearch - Text: combined_stemmed | Meta: False | Model: RandomForest
🔍 GridSearch - Text: combined_stemmed | Meta: False | Model: RandomForest
🔍 GridSearch - Text: combined_stemmed | Meta: False | Model: SVM
🔍 GridSearch - Text: combined_stemmed | Meta: False | Model: SVM
🔍 GridSearch - Text: combined_stemmed | Meta: False | Model: XGBoost
🔍 GridSearch - Text: combined_stemmed | Meta: False | Model: XGBoost


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:20:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:20:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:20:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:20:21] WARN

🔍 GridSearch - Text: combined_stemmed | Meta: True | Model: LogisticRegression
🔍 GridSearch - Text: combined_stemmed | Meta: True | Model: RandomForest
🔍 GridSearch - Text: combined_stemmed | Meta: True | Model: RandomForest
🔍 GridSearch - Text: combined_stemmed | Meta: True | Model: SVM
🔍 GridSearch - Text: combined_stemmed | Meta: True | Model: SVM
🔍 GridSearch - Text: combined_stemmed | Meta: True | Model: XGBoost
🔍 GridSearch - Text: combined_stemmed | Meta: True | Model: XGBoost


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:20:28] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:20:28] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:20:28] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:20:28] WARN


✅ Résultats sauvegardés dans gridsearch_results.csv


In [16]:


df_results = pd.read_csv("gridsearch_results.csv")


In [17]:
# Affiche les top modèles triés par f1-score sur les catastrophes
top_results = df_results.sort_values(by="f1_1", ascending=False)

# Affiche les 5 meilleurs
print(top_results[["text_column", "use_meta", "model", "best_params", "f1_1"]].head(5))


      text_column  use_meta               model      best_params      f1_1
0  combined_lemma     False  LogisticRegression  {'clf__C': 0.1}  0.782007
1           lemma     False  LogisticRegression    {'clf__C': 1}  0.777523
2      clean_text      True  LogisticRegression    {'clf__C': 1}  0.773585
3           lemma      True  LogisticRegression    {'clf__C': 1}  0.770318
4         stemmed      True  LogisticRegression    {'clf__C': 1}  0.769231


## 🔧 Correction : Stratégie de validation avec test final à 10%

**Problème identifié** : Votre split 80/20 ne réserve pas de test final.

**Solution** : Stratégie train/validation/test (70/20/10) :

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Charger les données
df = pd.read_csv("df_balanced.csv")

# 🎯 STRATÉGIE DE VALIDATION CORRIGÉE
# 1. Séparer d'abord le test final (10%)
X = df.drop("target", axis=1)
y = df["target"]

X_temp, X_test_final, y_temp, y_test_final = train_test_split(
    X, y, test_size=0.10, stratify=y, random_state=42
)

# 2. Diviser le reste en train (70%) et validation (20%)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.22, stratify=y_temp, random_state=42  # 0.22 * 0.90 = 0.20
)

print("📊 Répartition des données :")
print(f"Training set   : {len(X_train):,} échantillons ({len(X_train)/len(df)*100:.1f}%)")
print(f"Validation set : {len(X_val):,} échantillons ({len(X_val)/len(df)*100:.1f}%)")  
print(f"Test final     : {len(X_test_final):,} échantillons ({len(X_test_final)/len(df)*100:.1f}%)")
print(f"Total          : {len(df):,} échantillons")

# Vérification de la stratification
print(f"\n🎯 Distribution des classes :")
print(f"Training   : {y_train.value_counts(normalize=True).values}")
print(f"Validation : {y_val.value_counts(normalize=True).values}")
print(f"Test final : {y_test_final.value_counts(normalize=True).values}")

print(f"\n⚠️ IMPORTANT : Le test final ne sera utilisé QU'UNE SEULE FOIS avec le meilleur modèle !")

📊 Répartition des données :
Training set   : 3,082 échantillons (70.2%)
Validation set : 870 échantillons (19.8%)
Test final     : 440 échantillons (10.0%)
Total          : 4,392 échantillons

🎯 Distribution des classes :
Training   : [0.5 0.5]
Validation : [0.5 0.5]
Test final : [0.5 0.5]

⚠️ IMPORTANT : Le test final ne sera utilisé QU'UNE SEULE FOIS avec le meilleur modèle !


## 🤖 Pipeline de modélisation avec Bagging & Boosting

Reprenons votre excellent pipeline avec la stratégie de validation corrigée :

In [19]:
# Configuration des expériences
TEXT_COLUMNS = {
    "clean_text": "clean_text", 
    "lemma": "text_lemma",                          
    "combined_lemma": "combined_lemma",
    "stemmed": "text_stem",                       
    "combined_stemmed": "combined_stemmed"
}

META_FEATURES = [
    "word_count", "stopword_count", "url_count", "char_count",
    "punctuation_count", "hashtag_count", "mention_count",
    "has_question", "exclamation_count", "presence_keyword", "presence_location"
]

# 🎯 FOCUS BAGGING & BOOSTING
MODELS_ENSEMBLE = {
    "RandomForest": (
        RandomForestClassifier(random_state=42),
        {
            "clf__n_estimators": [50, 100, 200],
            "clf__max_depth": [None, 10, 20],
            "clf__min_samples_split": [2, 5]
        }
    ),
    "XGBoost": (
        XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
        {
            "clf__n_estimators": [50, 100, 200],
            "clf__max_depth": [3, 6, 9],
            "clf__learning_rate": [0.01, 0.1, 0.2]
        }
    ),
    # Baseline pour comparaison
    "LogisticRegression": (
        LogisticRegression(max_iter=1000, random_state=42),
        {"clf__C": [0.1, 1, 10]}
    )
}

results_corrected = []

print("🚀 EXPÉRIENCES BAGGING & BOOSTING")
print("=" * 50)

# Boucle d'expérimentation
for text_name, text_col in TEXT_COLUMNS.items():
    for use_meta in [False, True]:
        selected_features = [text_col] + (META_FEATURES if use_meta else [])
        
        # Vérifier que toutes les colonnes existent
        if not all(col in df.columns for col in selected_features):
            print(f"⚠️ Colonnes manquantes pour {text_name}, use_meta={use_meta}")
            continue

        X_train_exp = X_train[selected_features]
        X_val_exp = X_val[selected_features]

        for model_name, (model, param_grid) in MODELS_ENSEMBLE.items():
            print(f"🔍 Text: {text_name} | Meta: {use_meta} | Model: {model_name}")

            # Configuration du preprocessing
            transformers = [("tfidf", TfidfVectorizer(max_features=3000, ngram_range=(1,2)), text_col)]
            if use_meta:
                transformers.append(("scaler", StandardScaler(), META_FEATURES))

            preprocessor = ColumnTransformer(transformers)
            
            # Pipeline complet
            pipe = Pipeline([
                ("preprocessing", preprocessor),
                ("clf", model)
            ])

            # GridSearch avec validation croisée
            grid = GridSearchCV(
                pipe, param_grid, 
                cv=3, 
                scoring='f1',  # Important pour les catastrophes
                n_jobs=-1, 
                verbose=0
            )
            
            # Entraînement
            grid.fit(X_train_exp, y_train)
            
            # Prédiction sur validation
            y_val_pred = grid.predict(X_val_exp)
            
            # Métriques
            report = classification_report(y_val, y_val_pred, output_dict=True)
            
            results_corrected.append({
                "text_column": text_name,
                "use_meta": use_meta,
                "model": model_name,
                "strategy": "Bagging" if model_name == "RandomForest" else 
                           "Boosting" if model_name == "XGBoost" else "Baseline",
                "best_params": grid.best_params_,
                "accuracy": report["accuracy"],
                "precision_1": report["1"]["precision"],
                "recall_1": report["1"]["recall"],
                "f1_1": report["1"]["f1-score"],
                "best_cv_score": grid.best_score_
            })

print("\n✅ Expériences terminées !")

# Résultats
df_results_corrected = pd.DataFrame(results_corrected)
df_results_corrected = df_results_corrected.sort_values(by="f1_1", ascending=False)

print("\n🏆 TOP 5 MODÈLES (triés par F1-score catastrophes) :")
display(df_results_corrected[["text_column", "use_meta", "model", "strategy", "f1_1", "recall_1", "accuracy"]].head())

🚀 EXPÉRIENCES BAGGING & BOOSTING
🔍 Text: clean_text | Meta: False | Model: RandomForest
🔍 Text: clean_text | Meta: False | Model: XGBoost


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:20:35] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:20:35] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:20:35] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:20:35] WARN

🔍 Text: clean_text | Meta: False | Model: LogisticRegression
🔍 Text: clean_text | Meta: True | Model: RandomForest
🔍 Text: clean_text | Meta: True | Model: XGBoost


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:20:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:20:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:20:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:20:58] WARN

🔍 Text: clean_text | Meta: True | Model: LogisticRegression
🔍 Text: lemma | Meta: False | Model: RandomForest
🔍 Text: lemma | Meta: False | Model: XGBoost


...

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:23:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:23:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:23:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:23:16] WARN

🔍 Text: combined_stemmed | Meta: False | Model: LogisticRegression
🔍 Text: combined_stemmed | Meta: True | Model: RandomForest
🔍 Text: combined_stemmed | Meta: True | Model: XGBoost


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:23:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:23:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:23:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [01:23:36] WARN

🔍 Text: combined_stemmed | Meta: True | Model: LogisticRegression

✅ Expériences terminées !

🏆 TOP 5 MODÈLES (triés par F1-score catastrophes) :


,text_column,use_meta,model,strategy,f1_1,recall_1,accuracy
23,stemmed,True,LogisticRegression,Baseline,0.784722,0.779310,0.786207
14,combined_lemma,False,LogisticRegression,Baseline,0.772622,0.765517,0.774713
20,stemmed,False,LogisticRegression,Baseline,0.770335,0.740230,0.779310
26,combined_stemmed,False,LogisticRegression,Baseline,0.769591,0.756322,0.773563
11,lemma,True,LogisticRegression,Baseline,0.768150,0.754023,0.772414


## 📊 Analyse comparative : Bagging vs Boosting

In [20]:
# Analyse par stratégie d'ensemble
strategy_analysis = df_results_corrected.groupby("strategy")[["accuracy", "precision_1", "recall_1", "f1_1"]].agg(['mean', 'std'])

print("🎯 PERFORMANCE MOYENNE PAR STRATÉGIE :")
print("=" * 50)
for strategy in ["Bagging", "Boosting", "Baseline"]:
    if strategy in strategy_analysis.index:
        metrics = strategy_analysis.loc[strategy]
        print(f"\n📈 {strategy.upper()}")
        print(f"  Accuracy : {metrics[('accuracy', 'mean')]:.3f} ± {metrics[('accuracy', 'std')]:.3f}")
        print(f"  Recall   : {metrics[('recall_1', 'mean')]:.3f} ± {metrics[('recall_1', 'std')]:.3f}")
        print(f"  F1-Score : {metrics[('f1_1', 'mean')]:.3f} ± {metrics[('f1_1', 'std')]:.3f}")

# Comparaison avec/sans méta-features
print(f"\n🔧 IMPACT DES MÉTA-FEATURES :")
meta_analysis = df_results_corrected.groupby("use_meta")[["f1_1", "recall_1"]].mean()
print(meta_analysis)

# Meilleur modèle par stratégie
print(f"\n🏆 MEILLEUR MODÈLE PAR STRATÉGIE :")
for strategy in ["Bagging", "Boosting"]:
    best_in_strategy = df_results_corrected[df_results_corrected["strategy"] == strategy].iloc[0]
    print(f"\n{strategy.upper()} - {best_in_strategy['model']}")
    print(f"  Text: {best_in_strategy['text_column']} | Meta: {best_in_strategy['use_meta']}")
    print(f"  F1: {best_in_strategy['f1_1']:.3f} | Recall: {best_in_strategy['recall_1']:.3f}")
    print(f"  Params: {best_in_strategy['best_params']}")

# Sélection du modèle champion
best_model_row = df_results_corrected.iloc[0]
print(f"\n🥇 MODÈLE CHAMPION SÉLECTIONNÉ :")
print(f"  {best_model_row['model']} ({best_model_row['strategy']})")
print(f"  Text: {best_model_row['text_column']} | Meta: {best_model_row['use_meta']}")
print(f"  F1-Score: {best_model_row['f1_1']:.3f}")
print(f"  Recall: {best_model_row['recall_1']:.3f}")

🎯 PERFORMANCE MOYENNE PAR STRATÉGIE :

📈 BAGGING
  Accuracy : 0.746 ± 0.020
  Recall   : 0.684 ± 0.035
  F1-Score : 0.729 ± 0.024

📈 BOOSTING
  Accuracy : 0.744 ± 0.011
  Recall   : 0.700 ± 0.018
  F1-Score : 0.732 ± 0.012

📈 BASELINE
  Accuracy : 0.773 ± 0.007
  Recall   : 0.752 ± 0.016
  F1-Score : 0.768 ± 0.007

🔧 IMPACT DES MÉTA-FEATURES :
             f1_1  recall_1
use_meta                   
False     0.74944  0.715402
True      0.73621  0.708506

🏆 MEILLEUR MODÈLE PAR STRATÉGIE :

BAGGING - RandomForest
  Text: combined_stemmed | Meta: False
  F1: 0.759 | Recall: 0.736
  Params: {'clf__max_depth': None, 'clf__min_samples_split': 5, 'clf__n_estimators': 200}

BOOSTING - XGBoost
  Text: stemmed | Meta: True
  F1: 0.749 | Recall: 0.724
  Params: {'clf__learning_rate': 0.2, 'clf__max_depth': 6, 'clf__n_estimators': 200}

🥇 MODÈLE CHAMPION SÉLECTIONNÉ :
  LogisticRegression (Baseline)
  Text: stemmed | Meta: True
  F1-Score: 0.785
  Recall: 0.779


## 🎯 Évaluation finale sur le test set (10%)

**⚠️ ATTENTION** : Cette évaluation ne doit être faite QU'UNE SEULE FOIS avec le modèle champion !

In [ ]:
# Ré-entraînement du modèle champion sur train+validation
print("🚀 ÉVALUATION FINALE DU MODÈLE CHAMPION")
print("=" * 50)
# Récupérer les paramètres du meilleur modèle
best_config = df_results_corrected.iloc[0]
text_col = best_config['text_column']
use_meta = best_config['use_meta']
model_name = best_config['model']
best_params = eval(best_config['best_params'])  # Attention : eval n'est pas sécurisé en production

print(f"📋 Configuration du modèle champion :")
print(f"  - Modèle : {model_name} ({best_config['strategy']})")
print(f"  - Texte : {text_col}")
print(f"  - Méta-features : {use_meta}")
print(f"  - Paramètres : {best_params}")

# Préparer les features
selected_features = [text_col] + (META_FEATURES if use_meta else [])
X_train_final = pd.concat([X_train, X_val])[selected_features]  # Train + Validation
y_train_final = pd.concat([y_train, y_val])
X_test_final_exp = X_test_final[selected_features]

# Recréer le pipeline avec les meilleurs paramètres
if model_name == "RandomForest":
    final_model = RandomForestClassifier(random_state=42)
elif model_name == "XGBoost":
    final_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
else:
    final_model = LogisticRegression(max_iter=1000, random_state=42)

# Appliquer les meilleurs paramètres
for param, value in best_params.items():
    if param.startswith('clf__'):
        setattr(final_model, param[5:], value)

# Configuration preprocessing
transformers = [("tfidf", TfidfVectorizer(max_features=3000, ngram_range=(1,2)), text_col)]
if use_meta:
    transformers.append(("scaler", StandardScaler(), META_FEATURES))

preprocessor = ColumnTransformer(transformers)
final_pipeline = Pipeline([
    ("preprocessing", preprocessor),
    ("clf", final_model)
])

# Entraînement final
print(f"\n🔧 Entraînement sur {len(X_train_final)} échantillons...")
final_pipeline.fit(X_train_final, y_train_final)

# Prédiction finale
print(f"🎯 Évaluation sur {len(X_test_final_exp)} échantillons de test...")
y_test_final_pred = final_pipeline.predict(X_test_final_exp)

# Rapport final
final_report = classification_report(y_test_final, y_test_final_pred, output_dict=True)
final_cm = confusion_matrix(y_test_final, y_test_final_pred)

print(f"\n🏆 RÉSULTATS FINAUX :")
print(f"  - Accuracy : {final_report['accuracy']:.3f}")
print(f"  - Precision (catastrophes) : {final_report['1']['precision']:.3f}")
print(f"  - Recall (catastrophes) : {final_report['1']['recall']:.3f}")
print(f"  - F1-Score (catastrophes) : {final_report['1']['f1-score']:.3f}")

print(f"\n📊 Matrice de confusion :")
print(f"  TN: {final_cm[0,0]} | FP: {final_cm[0,1]}")
print(f"  FN: {final_cm[1,0]} | TP: {final_cm[1,1]}")

print(f"\n✅ Évaluation finale terminée !")
print(f"⚠️ Ces résultats représentent la performance réelle estimée du modèle.")

In [21]:
df['clean_text']

0       bomb head explosive decisions dat produced mor...
1       okay not sure the word mass murder applies dur...
2       like why on earth would you want anybody to be...
3           my woman crush wedneday goes to the beautiful
4       many deaths in shipwreck rescuers are trying t...
5       metal cutting sparks brush fire in brighton a ...
6       video were picking up bodies from water rescue...
7              mike magner discusses a trust betrayed via
8       set a new record states in days i dont even kn...
9                  trouble trouble when i dont get my way
10      fire hazard associated with installation of no...
11      suspect in latest us theatre attack had psycho...
12          no harm no foul and somebody needed to say it
13                   im over here listening to bloody jay
14      tonight its going to be mayhem everybody free ...
15      today marks the th anniversary of the bombing ...
16      jacksonville family bands together as memorial...
17            